For each 24 hour day (midnight tomidnight) we'd like to know the following:
* Temperature and barometric pressure values at sunrise and sunset
* Difference between temp and pressure at sunrise and sunset
* Maximum, minimum and averages for temp and pressure for both of the time frames of sunset to sunrise and sunrise to sunset
* Differences between the max, min and average for both of these time frames.


# Load libraries

In [1]:
# pandas for data structure
import pandas as pd

# Load data

#### Weather Data

* data collected from [Wunderground](https://www.wunderground.com/weather/api/)
* hosted file: [Google Drive](https://drive.google.com/file/d/1eS0gGM14g7iFulUeqz3XwbKb5OtK9aSI/view)

In [5]:
# local file
filename_wunderground = '../data/wunderground-170701_171101-day_night.csv'

In [6]:
# load data into dataframes
wund = pd.read_csv(filename_wunderground)

In [29]:
wund.columns

Index(['utc_date', 'station_id', 'dewpti', 'dewptm', 'heatindexi',
       'heatindexm', 'hum', 'precip_ratei', 'precip_ratem', 'precip_totali',
       'precip_totalm', 'pressurei', 'pressurem', 'tempi', 'tempm', 'wdird',
       'wdire', 'wgusti', 'wgustm', 'windchilli', 'windchillm', 'wspdi',
       'wspdm'],
      dtype='object')

In [33]:
wund.tail(2)

,utc_date,station_id,dewpti,dewptm,heatindexi,heatindexm,hum,precip_ratei,precip_ratem,precip_totali,...,tempi,tempm,wdird,wdire,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm
727762,2017-11-01 05:51:00,KMTVICTO9,13.5,-10.3,-9999.0,-9999.0,22,0.0,0.0,0.0,...,50.9,10.5,0,North,5.4,8.7,-999.0,-999.0,1.3,2.1
727763,2017-11-01 05:56:00,KMTVICTO9,13.1,-10.5,-9999.0,-9999.0,22,0.0,0.0,0.0,...,50.5,10.3,0,North,4.0,6.4,-999.0,-999.0,1.3,2.1


#### Sunset Sunrise Data

In [25]:
# Load Sunset Sunrise data
sun_filename = '../data/sunrise_sunset-wunderground-utc.csv'
sun = pd.read_csv(sun_filename)

In [28]:
sun.columns

Index(['astronomical_twilight_begin', 'astronomical_twilight_end',
       'civil_twilight_begin', 'civil_twilight_end', 'date', 'day_length',
       'nautical_twilight_begin', 'nautical_twilight_end', 'solar_noon',
       'station_id', 'sunrise', 'sunset'],
      dtype='object')

In [34]:
sun.head(2)

,astronomical_twilight_begin,astronomical_twilight_end,civil_twilight_begin,civil_twilight_end,date,day_length,nautical_twilight_begin,nautical_twilight_end,solar_noon,station_id,sunrise,sunset
0,2017-06-30T09:05:17+00:00,2017-07-01T06:14:50+00:00,2017-06-30T11:09:41+00:00,2017-07-01T04:10:26+00:00,2017-06-30,56621,2017-06-30T10:17:42+00:00,2017-07-01T05:02:26+00:00,2017-06-30T19:40:04+00:00,KMTCORVA9,2017-06-30T11:48:13+00:00,2017-07-01T03:31:54+00:00
1,2017-07-01T09:06:37+00:00,2017-07-02T06:13:53+00:00,2017-07-01T11:10:21+00:00,2017-07-02T04:10:09+00:00,2017-07-01,56572,2017-07-01T10:18:31+00:00,2017-07-02T05:01:59+00:00,2017-07-01T19:40:15+00:00,KMTCORVA9,2017-07-01T11:48:49+00:00,2017-07-02T03:31:41+00:00


*add local time column*

# New DataFrame

### Column for values at Sunrise and Sunset
* 'values' refers to temperature and pressure data
* indexed by day
* date will go sunrise to sunset
* columns = ['rise_tempi','set_tempi','rise_pressurei','set_pressurei']
* index = ['2017-07-01', .... '2017-10-31']

### Difference sunset, sunrise values

### Max, min, ave : sunset to sunrise : sunrise to sunset

### Difference max, min, ave sunset to sunrise : sunrise to sunset

### Scratch

In [21]:
col = ['astronomical_twilight_begin','astronomical_twilight_end','civil_twilight_begin',
       'civil_twilight_end','date','day_length','nautical_twilight_begin','nautical_twilight_end',
       'solar_noon','station_id','sunrise','sunset']